In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import probnum as pn

import linpde_gp

In [ ]:
import experiment_utils
from experiment_utils import config

config.experiment_name = "0001_cpu_stationary_2d"
config.target = "jmlr"
config.debug_mode = True

## Problem Definition

### References
1. https://en.wikichip.org/wiki/intel/microarchitectures/coffee_lake#Quad-Core
2. https://ark.intel.com/content/www/us/en/ark/products/134896/intel-core-i59600k-processor-9m-cache-up-to-4-60-ghz.html
3. https://ark.intel.com/content/www/us/en/ark/products/134599/intel-core-i912900k-processor-30m-cache-up-to-5-20-ghz.html
4. https://journals.aps.org/pr/pdf/10.1103/PhysRev.134.A1058

### Geometry

In [ ]:
w_CPU = 9.19  # mm, [1]
l_CPU = 16.0  # ≈ 16.28 mm, [1]
d_CPU = 0.37  # mm, datasheed linked under [3] ("Supplemental Information / Datasheet")

A_CPU_top = l_CPU * w_CPU  # mm^2
A_CPU_EW = w_CPU * d_CPU  # mm^2
A_CPU_NS = l_CPU * d_CPU  # mm^2
A_CPU_TI = A_CPU_top + 2 * A_CPU_EW + 2 * A_CPU_NS  # mm^2, thermal interface area

V_CPU = w_CPU * l_CPU * d_CPU  # mm^3

In [ ]:
domain = linpde_gp.domains.Box([[0, l_CPU], [0, w_CPU]])

In [ ]:
X_core_centers = np.array([0.2, 0.4, 0.6]) * l_CPU  # estimated from the schematic in [1]
Y_core_centers = np.array([0.25, 0.75]) * w_CPU  # estimated from the schematic in [1]

core_centers = np.stack(np.meshgrid(X_core_centers, Y_core_centers), axis=-1)

### Material Properties

In [ ]:
# Thermal conductivity
k_CPU = 1.56 # W/cm K, [4] TODO: Improve this estimate
k_CPU *= 10  # W/mm K

### Heat Sources and Heat Sinks

In [ ]:
TDP_CPU = 95 # W, [2]

In [ ]:
# CPU Cores = Heat Sources
core_heat_std = np.array([
    0.09 * l_CPU,
    0.15 * w_CPU,
])

f_CPU = TDP_CPU / d_CPU * pn.LambdaFunction(
    lambda x: (
        1.0 / (np.sqrt(2 * np.pi) * np.prod(core_heat_std))
        * np.mean(
            np.exp(
                -0.5 * np.sum(
                    ((x[..., None, None, :] - core_centers) / core_heat_std) ** 2,
                    axis=-1
                )
            ),
            axis=(-2, -1),
        )
    ),
    input_shape=(2,),
    output_shape=(),
)

In [ ]:
# CPU cooler = "Uniform Heat Sink"
f_cooler = linpde_gp.functions.Constant(
    input_shape=(2,),
    value=TDP_CPU / A_CPU_top / d_CPU,
)

In [ ]:
# Total amount of heat entering/leaving the CPU
f = f_CPU - f_cooler

In [ ]:
# %matplotlib widget
plt_grid_x, plt_grid_y = np.meshgrid(
    np.linspace(*domain[0], 100),
    np.linspace(*domain[1], 100),
)
plt_grid = np.stack((plt_grid_x, plt_grid_y), axis=-1)

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
ax.plot_surface(plt_grid_x, plt_grid_y, f_CPU(plt_grid))
# plt.imshow(f(plt_grid))

### PDE

In [ ]:
diffop = -k_CPU * linpde_gp.linfuncops.diffops.Laplacian(domain.shape)

In [ ]:
u_prior = pn.randprocs.GaussianProcess(
    mean=linpde_gp.functions.Constant(input_shape=(2,), value=59.0),
    cov=3.0 ** 2 * linpde_gp.randprocs.kernels.ProductMatern(
        input_shape=(2,),
        p=3,
        lengthscales=[8.0, 8.0],
    ),
)

In [ ]:
N_pde = 10

X_pde = np.stack(
    np.meshgrid(
        np.linspace(
            domain[0][0] + 0.03 * l_CPU,
            domain[0][1] - 0.03 * l_CPU,
            N_pde,
        ),
        np.linspace(
            domain[0][0] + 0.03 * l_CPU,
            domain[0][1] - 0.03 * l_CPU,
            N_pde,
        ),
    ),
    axis=-1,
)

In [ ]:
u_cond_pde = u_prior.condition_on_observations(
    Y=np.zeros_like(X_pde, shape=X_pde.shape[:-1]),
    X=X_pde,
    L=diffop,
    b=-f(X_pde),
)

In [ ]:
plt_grid_x, plt_grid_y = np.meshgrid(
    np.linspace(*domain[0], 100),
    np.linspace(*domain[1], 100),
)
plt_grid = np.stack((plt_grid_x, plt_grid_y), axis=-1)

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
ax.plot_surface(plt_grid_x, plt_grid_y, u_cond_pde.mean(plt_grid))